# This is the main file please don't tamper with it

## Import the required libraries

In [35]:
import tweepy
import csv
import time
import glob
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import re

## Twitter API credentials

In [8]:
consumer_key = "4xihrxvf1Sbuaufe7RWNNGBQ4c"
consumer_secret = "wyzB7mZvxzlFOUFakETiV5USLStSX0zEaQP4qinU4QUnOjWeZxi"
access_key = "2x306144275-Oc7NnFdC7H9v5BjAysBz7si4OWZ9N7xmIc6XbLl"
access_secret = "UFLgsZzTYptlDTQUjjLfhG4tZzVvPkgYDVuMPitYOAVFxA"

## Authorize twitter, initialize tweepy

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

## Create a list of all counties for which we need tweets. We will iterate through this list

In [10]:
counties = ["BondCountyHD", "boonecohealth", "CU_PublicHealth", "ChiPublicHealth", "CookCtyHealth", "DeKalbCoHD", "DuPageHD", 
            "fwbicohealth", "KaneCoHealth", "kankakeehealth", "kendallhealth", "kchd92", "LakeCoHealth", "LaSalleCoHealth", "LCHD_IL", 
            "LCHD_Illinois", "Maconcountyhd", "MacoupinHealth", "McHenryCoHealth", "McLeanHealth", "vopnews", 
            "peoriaprepare", "polkcohealth", "RICO_HealthDept", "HenryStarkHD", "stclairhealth", "ERCSCHD", 
            "tazewellhealth", "Whiteside_CHC", "WillCoHealth", "WinnCoHealth", "WoodfordHealth"]

## Function to retrieve all hashtags in a tweet

In [19]:
def getHashtags(tweet):
    #hashtags=[None]*len(tweet["entities"]["hashtags"])
    hashtags=[]
    if len(tweet["entities"]["hashtags"]) > 0:
        for i in range(len(tweet["entities"]["hashtags"])-1):
            hashtags.append(tweet["entities"]["hashtags"][i]['text'])
        #print(hashtags)
        hashtags=list(set(hashtags))
        hashtags=' '.join(hashtags)
    return hashtags

## Function to know if the tweet has image/video/etc

In [12]:
def getMedia(tweet):
    try:
        value = tweet['extended_entities']['media'][0]['type']
        return value
    except KeyError:
        return "NA"

## Function to get date time in a proper format

In [13]:
def getDate(tweet):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))

## Function to get tweet source from HTML format (Get value of anchor tag)

In [14]:
def getTweetSource(tweet):
    #Beautiful Soup is used here
    soup=BS(tweet)
    return soup.find('a', {'rel':'nofollow'}).text

## Function to know if it is a retweet

In [26]:
def getTweetType(tweet):
    if re.search('^RT\s@*', tweet):
        return "Retweet"
    else:
        return "Tweet"

## Function to get URL of a tweet

In [70]:
def getTweetURL(tweet):
    url = "twitter.com/" + tweet['user']['screen_name'] + "/status/" + tweet['id_str']
    return url

## Iterating through the list and generating a csv for each county

In [71]:
for county in counties:
    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    new_tweets = api.user_timeline(screen_name = county,count=200, parser=tweepy.parsers.JSONParser(), tweet_mode = "extended")

    #save most recent tweets
    alltweets.extend(new_tweets)
    
    
    oldest = alltweets[-1]['id'] - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:

        
        #print("getting tweets before %s" % (oldest))
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = county,count=200,max_id=oldest, parser=tweepy.parsers.JSONParser(), 
                                       tweet_mode = "extended")

        #save most recent tweets
        alltweets.extend(new_tweets)

         #update the id of the oldest tweet less one
        oldest = alltweets[-1]['id'] - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

        #transform the tweepy tweets into a 2D array that will populate the csv	
        outtweets = [[tweet['id_str'], getDate(tweet), tweet['full_text'], getTweetType(tweet['full_text']), 
                      tweet['user']['screen_name'], tweet['user']['location'],  
                      tweet['user']['followers_count'], tweet['user']['friends_count'], tweet['user']['favourites_count'], 
                      tweet['user']['statuses_count'], tweet['retweet_count'], tweet['favorite_count'], 
                      tweet['favorited'], getHashtags(tweet), getTweetSource(tweet['source']), getMedia(tweet),
                      getTweetURL(tweet)] for tweet in alltweets]

        #write the csv
        with open('%s_tweets.csv' % county, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id","created_at","tweet", "Tweet_Type", "twitter_handle", "location","followers", 
                             "following", "profile_likes", "total_tweets", "retweets", "favorites", "is_favorited",
                             "Hashtags", "source", "is_media", "Tweet_URL"])
            writer.writerows(outtweets)
    print(county," County is done----------------------------------------------------------")

...400 tweets downloaded so far


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


...600 tweets downloaded so far
...800 tweets downloaded so far
...1000 tweets downloaded so far
...1200 tweets downloaded so far
...1224 tweets downloaded so far
...1224 tweets downloaded so far
BondCountyHD  County is done----------------------------------------------------------
...400 tweets downloaded so far
...600 tweets downloaded so far
...603 tweets downloaded so far
...603 tweets downloaded so far
boonecohealth  County is done----------------------------------------------------------
...400 tweets downloaded so far
...600 tweets downloaded so far
...799 tweets downloaded so far
...999 tweets downloaded so far
...1199 tweets downloaded so far
...1396 tweets downloaded so far
...1596 tweets downloaded so far
...1796 tweets downloaded so far
...1996 tweets downloaded so far
...2196 tweets downloaded so far
...2264 tweets downloaded so far
...2264 tweets downloaded so far
CU_PublicHealth  County is done----------------------------------------------------------
...400 tweets downl

...2432 tweets downloaded so far
...2432 tweets downloaded so far
peoriaprepare  County is done----------------------------------------------------------
...400 tweets downloaded so far
...600 tweets downloaded so far
...800 tweets downloaded so far
...1000 tweets downloaded so far
...1200 tweets downloaded so far
...1400 tweets downloaded so far
...1600 tweets downloaded so far
...1800 tweets downloaded so far
...2000 tweets downloaded so far
...2200 tweets downloaded so far
...2399 tweets downloaded so far
...2599 tweets downloaded so far
...2799 tweets downloaded so far
...2999 tweets downloaded so far
...3199 tweets downloaded so far
...3215 tweets downloaded so far
...3215 tweets downloaded so far
polkcohealth  County is done----------------------------------------------------------
...74 tweets downloaded so far
RICO_HealthDept  County is done----------------------------------------------------------
...353 tweets downloaded so far
...353 tweets downloaded so far
HenryStarkHD  Co

## Combining all csv files

In [72]:
csvs = glob.glob('*.csv')

In [73]:
len(csvs)

32

In [74]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in csvs])
#export to csv
combined_csv.to_csv( "ALLTWEETS.csv", index=False, encoding='utf-8-sig')

## Shuffling all rows and assigning tweets to each group member

In [75]:
df= pd.read_csv('ALLTWEETS.csv')

## Backing up the original data frame

In [76]:
original = df

## Shuffling the data frame rows

In [77]:
df_n=df.sample(frac=1)

## We need to consider only tweets by hospitals and not retweets

In [78]:
df_n=df_n[df_n['Tweet_Type']=='Tweet']

## Assigning top 800 rows to 8 members (100 per member)

In [79]:
df_n['Team_Member']='Default'

In [80]:
df_n['Team_Member'][0:100]='Devansh'
df_n['Team_Member'][101:200]='Adithi'
df_n['Team_Member'][201:300]='Shweta'
df_n['Team_Member'][301:400]='Anuj'

df_n['Team_Member'][401:500]='Kalyani'
df_n['Team_Member'][501:600]='Sanjana'
df_n['Team_Member'][601:700]='Neil'
df_n['Team_Member'][701:800]='Nupur'

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_ne

## Save as data frame

In [81]:
df_n.to_csv("ALL_TWEETS_V2.csv", index=False)